In [ ]:
# Setup CustomVision credentials

from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

endpoint = 'ENDPOINT'
prediction_key = 'PREDICTION_KEY'
project_id = 'PROJECT_ID'
iteration_id = 'ITERATION_ID'
predictor = CustomVisionPredictionClient(prediction_key, endpoint=endpoint)

In [ ]:
# Load prerequisits 

import mPyPl as mp
import json 
from os.path import join
from metrics import *
from PIL import Image

data_dir = '../data/Tagged'
vott_export = json.load(open('../data/Tagged.json', 'r'))
test_images = json.load(open('test_images.json', 'r'))
test_images = test_images['test_images']
input_tags = vott_export['inputTags'].split(',')

In [ ]:
# Define some useful functions 

def ignore_result(func, value):
    return func or value

def box_to_whformat(box):
    box['width'] = abs(box['x2'] - box['x1'])
    box['height'] = abs(box['y2'] - box['y1'])
    box.pop('x2', None)
    box.pop('y2', None)
    return box

def prediction_as_dict(prediction, width=None, height=None):
    return {
        'tag' : prediction.tag_name,
        'prob' : prediction.probability,
        'x1' : prediction.bounding_box.left * width if width else prediction.bounding_box.left,
        'y1' : prediction.bounding_box.top * height if height else prediction.bounding_box.top,
        'width' : prediction.bounding_box.width * width if width else prediction.bounding_box.width,
        'height' : prediction.bounding_box.height * height if height else prediction.bounding_box.height
    }

def format_dict(prediction, width=None, height=None):
    return {
        'tag' : prediction['tagName'],
        'prob' : prediction['probability'],
        'x1' : prediction['boundingBox']['left'] * width if width else prediction['boundingBox']['left'],
        'y1' : prediction['boundingBox']['top'] * height if height else prediction['boundingBox']['top'],
        'width' : prediction['boundingBox']['width'] * width if width else prediction['boundingBox']['width'],
        'height' : prediction['boundingBox']['height'] * height if height else prediction['boundingBox']['height']
    }

def print_report(stream, input_tags, pred_field='predictions', gt_field='ground_truth'):
    top = '{:15.12} | {:^12.10} | {:^12.10}'.format("Tag", "Precision", "Recall")
    print(top)
    print('-' * len(top))
    for tag in input_tags:
        precision, recall = precision_recall(stream, tag, 0.5, 0.3, pred_field=pred_field, gt_field=gt_field)
        print('{:15.12} | {:^12.5} | {:^12.5}'.format(tag, float(precision), float(recall)))

In [ ]:
# Compress CoreML model

import coremltools

# Option 1: convert a full precision (float) MLModel to a 16bit quantized MLModel
model_spec = coremltools.utils.load_spec('coreml/model.mlmodel')
model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
coremltools.utils.save_spec(model_fp16_spec, 'coreml/modelFP16.mlmodel')

# Option 2: convert a full precision (float) MLModel to a 8bit quantized MLModel
model = coremltools.models.MLModel('coreml/model.mlmodel')
model_fp8 = coremltools.models.neural_network.quantization_utils.quantize_weights(model, nbits=8)
coremltools.utils.save_spec(model_fp8, 'coreml/modelFP8.mlmodel')

In [ ]:
# Setup CoreML models

from coreml.python.predict import *

coreml_16 = coremltools.models.MLModel('coreml/modelFP16.mlmodel')
coreml_8 = coremltools.models.MLModel('coreml/modelFP8.mlmodel')
coreml_orig = coremltools.models.MLModel('coreml/model.mlmodel')

with open('coreml/labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]

od_coreml_16 = CoreMLObjectDetection(coreml_16, labels)
od_coreml_8 = CoreMLObjectDetection(coreml_8, labels)
od_coreml_orig = CoreMLObjectDetection(coreml_orig, labels)

In [ ]:
# Compress TensorFlow model
import tensorflow as tf

converter = tf.contrib.lite.TFLiteConverter.from_frozen_graph('tflite/model.pb', ['Placeholder'], ['model_outputs'])
converter.post_training_quantize = True
tflite_model_8 = converter.convert()
open("tflite/modelFP8.tflite", "wb").write(tflite_model_8)

In [ ]:
# Setup TensorFlow Original Model

from tflite.python.predict import TFObjectDetection as OrigTFObjectDetection
from tflite.python.predict_lite import TFObjectDetection as LiteTFObjectDetection

graph_orig = tf.GraphDef()
with tf.gfile.FastGFile('tflite/model.pb', 'rb') as f:
    graph_orig.ParseFromString(f.read())
        
# Load labels
with open('tflite/labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]

od_tflite_orig = OrigTFObjectDetection(graph_orig, labels)

od_tflite_8 = LiteTFObjectDetection('tflite/modelFP8.tflite', labels)

In [ ]:
# Setup datastream

stream = (
    list(vott_export['frames'].keys()) 
    #| mp.where(lambda x: x in test_images)
    | mp.as_field('filename')
    | mp.apply('filename', 'meta', lambda x: vott_export['frames'][x])
    | mp.apply('meta', 'width', lambda x: x[0]['width'])
    | mp.apply('meta', 'height', lambda x: x[0]['height'])
    | mp.apply('meta', 'ground_truth', lambda x: x 
        | mp.select(lambda m: ignore_result(m['box'].update({'tag' : m['tags'][0]}), m['box'])) 
        | mp.select(lambda m: box_to_whformat(m))
        | mp.as_list
     )
    # Predict using online CustomVision
    | mp.apply('filename', 'raw_cv', 
        lambda x: predictor.predict_image(project_id, open(join(data_dir, x), mode="rb"), iteration_id).predictions)
    | mp.apply(['raw_cv', 'width', 'height'], 'cv_predictions', lambda x: x[0]
        | mp.select(lambda p: prediction_as_dict(p, x[1], x[2]))
        | mp.as_list
      )     
    # Predict using CoreML original
    | mp.apply('filename', 'coreml_orig_raw', lambda x: od_coreml_orig.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['coreml_orig_raw', 'width', 'height'], 'coreml_orig_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      )  
    # Predict using CoreML compressed to 16FP
    | mp.apply('filename', 'coreml_16_raw', lambda x: od_coreml_16.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['coreml_16_raw', 'width', 'height'], 'coreml_16_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      )  
    # Predict using CoreML compressed to 8FP
    | mp.apply('filename', 'coreml_8_raw', lambda x: od_coreml_8.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['coreml_8_raw', 'width', 'height'], 'coreml_8_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      )  
    # Predict using TensorFlow original
    | mp.apply('filename', 'tf_orig_raw', lambda x: od_tflite_orig.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['tf_orig_raw', 'width', 'height'], 'tf_orig_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      ) 
    # Predict using TensorFlow compressed to 8FP
    | mp.apply('filename', 'tf_8_raw', lambda x: od_tflite_8.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['tf_8_raw', 'width', 'height'], 'tf_8_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      ) 
    | mp.delfield(['meta', 'raw_cv', 'coreml_orig_raw', 'coreml_16_raw', 'coreml_8_raw', 'tf_orig_raw', 'tf_8_raw'])
    | mp.as_list
)

stream[0]

In [ ]:
import pickle

pickle.dump(stream, open('results.pickle', 'wb'))
# stream = pickle.load(open('results.pickle', 'rb'))

In [ ]:
# Print reports

for pred_field in [key for key in stream[0].keys() if 'predictions' in key]:
    print("\n===== %s =====\n" % pred_field)
    print_report(stream, input_tags, pred_field=pred_field, gt_field='ground_truth')